# Data Cleaning and Aggregation - Topics and Subject Areas

### Import Packages and Prerequisites

In [ ]:
import pandas as pd
import numpy as np
import json
import ast

from datetime import datetime

pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.set_option('display.max_columns', None)

sfu_works = pd.read_csv('../data_pulls/sfu_all_works.csv')


In [ ]:
works_json = sfu_works.to_json()

done


### 1. Get the subject areas

In [ ]:
# isolate topics, subfields, fields, and domains
sfu_work_topic = sfu_works[['publication_year','primary_topic']]

# extract the topic, subfield, field, and domain
sfu_work_topic_cleaned = pd.DataFrame(sfu_work_topic.dropna().reset_index()[['publication_year','primary_topic']])
sfu_work_topic_cleaned['as_dict'] = sfu_work_topic_cleaned['primary_topic'].apply(ast.literal_eval)
sfu_work_topic_cleaned['open_alex_id'] = sfu_work_topic_cleaned['as_dict'].apply(lambda x: x['id'])
sfu_work_topic_cleaned['topic'] = sfu_work_topic_cleaned['as_dict'].apply(lambda x: x['display_name'])
sfu_work_topic_cleaned['subfield'] = sfu_work_topic_cleaned['as_dict'].apply(lambda x: x['subfield']).apply(lambda y: y['display_name'])
sfu_work_topic_cleaned['field'] = sfu_work_topic_cleaned['as_dict'].apply(lambda x: x['field']).apply(lambda y: y['display_name'])
sfu_work_topic_cleaned['domain'] = sfu_work_topic_cleaned['as_dict'].apply(lambda x: x['domain']).apply(lambda y: y['display_name'])

# print
sfu_work_topic_cleaned


,publication_year,primary_topic,as_dict,open_alex_id,topic,subfield,field,domain
0,2012,"{'id': 'https://openalex.org/T11106', 'display...","{'id': 'https://openalex.org/T11106', 'display...",https://openalex.org/T11106,Data Management and Algorithms,Signal Processing,Computer Science,Physical Sciences
1,2020,"{'id': 'https://openalex.org/T10095', 'display...","{'id': 'https://openalex.org/T10095', 'display...",https://openalex.org/T10095,Cosmology and Gravitation Theories,Astronomy and Astrophysics,Physics and Astronomy,Physical Sciences
2,2005,"{'id': 'https://openalex.org/T10302', 'display...","{'id': 'https://openalex.org/T10302', 'display...",https://openalex.org/T10302,Fish Ecology and Management Studies,Nature and Landscape Conservation,Environmental Science,Physical Sciences
3,2000,"{'id': 'https://openalex.org/T10538', 'display...","{'id': 'https://openalex.org/T10538', 'display...",https://openalex.org/T10538,Data Mining Algorithms and Applications,Information Systems,Computer Science,Physical Sciences
4,1994,"{'id': 'https://openalex.org/T10015', 'display...","{'id': 'https://openalex.org/T10015', 'display...",https://openalex.org/T10015,Genomics and Phylogenetic Studies,Molecular Biology,"Biochemistry, Genetics and Molecular Biology",Life Sciences
...,...,...,...,...,...,...,...,...
74912,1988,"{'id': 'https://openalex.org/T10588', 'display...","{'id': 'https://openalex.org/T10588', 'display...",https://openalex.org/T10588,Mathematical Dynamics and Fractals,Mathematical Physics,Mathematics,Physical Sciences
74913,1984,"{'id': 'https://openalex.org/T11859', 'display...","{'id': 'https://openalex.org/T11859', 'display...",https://openalex.org/T11859,Educator Training and Historical Pedagogy,Sociology and Political Science,Social Sciences,Social Sciences
74914,1993,"{'id': 'https://openalex.org/T10049', 'display...","{'id': 'https://openalex.org/T10049', 'display...",https://openalex.org/T10049,Magnetic properties of thin films,"Atomic and Molecular Physics, and Optics",Physics and Astronomy,Physical Sciences
74915,1989,"{'id': 'https://openalex.org/T10181', 'display...","{'id': 'https://openalex.org/T10181', 'display...",https://openalex.org/T10181,Natural Language Processing Techniques,Artificial Intelligence,Computer Science,Physical Sciences


### 2. Aggregate and group by subject area

In [23]:
sfu_topics_agg = pd.DataFrame(sfu_work_topic_cleaned.groupby(['publication_year', 'topic']).count()['open_alex_id'].sort_values(ascending=False)).rename(columns={'open_alex_id': 'count'})
sfu_subfields_agg = pd.DataFrame(sfu_work_topic_cleaned.groupby(['publication_year', 'subfield']).count()['open_alex_id'].sort_values(ascending=False)).rename(columns={'open_alex_id': 'count'})
sfu_fields_agg = pd.DataFrame(sfu_work_topic_cleaned.groupby(['publication_year', 'field']).count()['open_alex_id'].sort_values(ascending=False)).rename(columns={'open_alex_id': 'count'})
sfu_domains_agg = pd.DataFrame(sfu_work_topic_cleaned.groupby(['publication_year', 'domain']).count()['open_alex_id'].sort_values(ascending=False)).rename(columns={'open_alex_id': 'count'})

### 3. Save in .csv format

In [ ]:
sfu_topics_agg.to_csv('../data_pulls/sfu_works_by_topic.csv')
sfu_subfields_agg.to_csv('../data_pulls/sfu_works_by_subfield.csv')
sfu_fields_agg.to_csv('../data_pulls/sfu_works_by_field.csv')
sfu_domains_agg.to_csv('../data_pulls/sfu_works_by_domain.csv')

### BONUS: Calculate institutional FWCI while we're already here

In [25]:
all_fwcis = sfu_works[['publication_year','fwci']]
all_fwcis

,publication_year,fwci
0,2012,1728.3
1,2020,676.1
2,2005,17.3
3,2000,276.0
4,1994,12.6
...,...,...
76588,1984,0.0
76589,1993,0.0
76590,1989,0.0
76591,1989,0.0


In [ ]:
current_yr = datetime.now().year

sfu_fwci_5yr = np.mean(all_fwcis[all_fwcis['publication_year'] > (current_yr-5)]['fwci'])
sfu_fwci_10yr = np.mean(all_fwcis[all_fwcis['publication_year'] > (current_yr-10)]['fwci'])
total_sfu_fwci = np.mean(all_fwcis['fwci'].dropna())

summary = pd.DataFrame(pd.read_csv('../data_pulls/sfu_summary_stats.csv'))
summary.loc[summary['Time Frame']=='5 years', 'Field-Weighted Citation Impact'] = sfu_fwci_5yr
summary.loc[summary['Time Frame']=='10 years', 'Field-Weighted Citation Impact'] = sfu_fwci_10yr
summary.loc[summary['Time Frame']=='All-time', 'Field-Weighted Citation Impact'] = total_sfu_fwci

summary.to_csv('../data_pulls/sfu_summary_stats.csv', index=False)

  Time Frame  Publications  Citations  Citations per Publication  h-Index  \
0   All-time       78135.0  2593163.0                       33.2    491.0   
1   10 years           NaN        NaN                        NaN      NaN   
2    5 years           NaN        NaN                        NaN      NaN   

  Field-Weighted Citation Impact        Authors  
0                            2.8  dont know yet  
1                            2.7            NaN  
2                            2.1            NaN  


### BONUS: Calculate publications per year

In [27]:
sfu_works_by_year = pd.DataFrame(sfu_works.groupby(['publication_year']).count()['id']).rename(columns={'id': 'Publications'})
sfu_works_by_year.to_csv('../data_pulls/sfu_works_by_year.csv', index=True)
sfu_works_by_year # this exists altready but this one has a longer time scope (but some minor inconsistencies)

,Publications
publication_year,
1964,1
1965,4
1966,27
1967,57
1968,69
...,...
2021,3707
2022,3406
2023,3525
